<h1> Data cropping </h1> <br> A jupyter notebook document for loading volumes and cropping them into smaller volumes with overlap <br> <br> <b>Functions</b> <br> load_single_image: Loads a raw volume and corresponding segmentation <br> <b>Parameters</b> <br> user_id: String containing special signature of user placement in cluster <br> data_path: Path to raw volume <br> segment_path: Path to corresponding segmentation <br> <br> Crop volume:  Function for dividing raw volume and segmentation into smaller crops with overlap <br> <b>Parameters</b> <br>
root_data: The variable that contains the volume <br> header: Meta data of the image <br> patch_size The size of each subvolume <br> output_dir The directory the outputs will be placed in <br> name: root file name <br> k: Overlap factor of subvolumes <br> no_root: A boolean flag that, if set to True, ensures only subvolumes containing roots are saved (based on the segmentation labels). If False, it saves all cropped subvolumes.

In [1]:
import os
import numpy as np
import nibabel as nib
import torch
from pathlib import Path
import qim3d
from os.path import join

In [ ]:

patch_size = (400,400,400)
overlap = 0.1


#IO parameters
user_id = str(Path.home())
input_parent_folder = "/QIM/projects/2024_DANFIX_147_RAPID-P/analysis/BachelorProject/Data/Downsampled/NonCropped/Week2/"
input_volume = "week2_30_top.nii.gz"
input_segmentation = "week2_30_top_root-class.nii.gz"


output_parent_folder = "QIM/projects/2024_DANFIX_147_RAPID-P/analysis/BachelorProject/Data/Downsampled/Cropped/"
output_folder_volume = "Week2/Week2_30_top/"
output_folder_segmentation = "Segmentations/Week2_30_top_1/"

name_root = "week2_30_top_1"

patch_size_str = "_".join(map(str, patch_size))
name = name_root + "_" + patch_size_str + "_" + str(overlap)

data_path = input_parent_folder+input_volume
segmentation_path = input_parent_folder+input_segmentation

data_output = output_parent_folder+output_folder_volume
segmentation_output = output_parent_folder+output_folder_segmentation


week2_30_top_1_400_400_400_0.1


In [ ]:
#Function definitions

# Function to load a single image
def load_single_image(user_id, data_path,segment_path):
    data_file = user_id + data_path
    segment_file = user_id + segment_path
    root_data, data_header = qim3d.io.load(data_file, return_metadata=True)
    root_labels, label_header = qim3d.io.load(segment_file, return_metadata=True)
    root_data = torch.from_numpy(root_data).float()
    root_labels = torch.from_numpy(root_labels).float()
    return root_data, data_header, root_labels, label_header

# Function to crop the volume
def crop_volume(root_data, header, patch_size, output_dir, name, k, user_id, no_root=False):
    """
    This function crops a 3D volume into smaller subvolumes (patches) and saves them.

    Parameters:
    - root_data: 3D tensor or array of the root data (image).
    - header: Metadata of the image.
    - patch_size: Size of the patch (tuple of depth, height, width).
    - output_dir: Directory to save the cropped subvolumes.
    - name: Prefix for naming the cropped subvolumes.
    - k: Fraction of the patch size for the overlap (e.g., 0.2 for 20% overlap).
    - user_id: Directory path to the user's data to include in the filenames.
    - no_root: Boolean flag to only save patches with root (default is False).
    """
    
    
    overlap_size = [int(p * k) for p in patch_size]
    
    imgSpace = header['pixdim'][1:4]
    origin = np.array([0, 0, 0])
    affine = np.zeros((4, 4))
    affine[0:3, 3] = origin
    affine[0, 0] = imgSpace[0]
    affine[1, 1] = imgSpace[1]
    affine[2, 2] = imgSpace[2]
    affine[3, 3] = 1
    step = 0
    cropped_count = 0  

    for z in range(0, len(root_data[0]), patch_size[0] - overlap_size[0]):
        for y in range(0, len(root_data[1]), patch_size[1] - overlap_size[1]):
            for x in range(0, len(root_data[2]), patch_size[2] - overlap_size[2]):
                z_begin = max(z, 0)
                y_begin = max(y, 0)
                x_begin = max(x, 0)
                z_end = min(z + patch_size[0], root_data.shape[0])
                y_end = min(y + patch_size[1], root_data.shape[1])
                x_end = min(x + patch_size[2], root_data.shape[2])

                patch_label = root_data[z_begin:z_end, y_begin:y_end, x_begin:x_end]
                patch_label = patch_label.numpy().astype(np.float32)
                print(f"Patch Size for step {step}: {patch_label.shape} Coordinates Z:({z_begin}, {z_end}), Y:({y_begin}, {y_end}), X:({x_begin}, {x_end})")

                cropped_data_nib = nib.Nifti1Image(patch_label, affine)

                if no_root:
                    if np.any(patch_label == 1):  
                        img_filename = os.path.join(user_id, output_dir, f"{name}_cropped_w_root_{step}.nii.gz")
                        nib.save(cropped_data_nib, img_filename)
                        print(f"Cropped volume with roots saved successfully: {img_filename}")
                else:
                    img_filename = os.path.join(user_id,output_dir, f"{name}_cropped_{step}.nii.gz")
                    nib.save(cropped_data_nib, img_filename)
                    print(f"Cropped volume saved successfully: {img_filename}")

                step += 1
                cropped_count += 1  

    return cropped_count

In [ ]:
os.makedirs(os.path.join(data_output, user_id), exist_ok=True)
os.makedirs(os.path.join(segmentation_output, user_id), exist_ok=True)

print("Loading volume and segmentation")
root_data, data_header, root_labels, label_header = load_single_image(user_id, data_path, segmentation_path)

cropped_data_count = crop_volume(root_data, data_header, patch_size, data_output, name+"_data", overlap, user_id)
print(f"Total cropped data subvolumes: {cropped_data_count}")

cropped_labels_count = crop_volume(root_labels, label_header, patch_size, segmentation_output, name+"_segmentation", overlap, user_id, no_root=False)
print(f"Total cropped label subvolumes: {cropped_labels_count}")

Loading volume and segmentation
Cropping volume
Patch Size for step 0: (400, 400, 400) Coordinates Z:(0, 400), Y:(0, 400), X:(0, 400)
Cropped volume saved successfully: /zhome/4f/d/187167/QIM/projects/2024_DANFIX_147_RAPID-P/analysis/BachelorProject/Data/Downsampled/Cropped/Week2/Week2_30_top/week2_30_top_1_400_400_400_0.1_data_cropped_0.nii.gz
Patch Size for step 1: (400, 400, 400) Coordinates Z:(0, 400), Y:(0, 400), X:(360, 760)
Cropped volume saved successfully: /zhome/4f/d/187167/QIM/projects/2024_DANFIX_147_RAPID-P/analysis/BachelorProject/Data/Downsampled/Cropped/Week2/Week2_30_top/week2_30_top_1_400_400_400_0.1_data_cropped_1.nii.gz
Patch Size for step 2: (400, 400, 180) Coordinates Z:(0, 400), Y:(0, 400), X:(720, 900)
Cropped volume saved successfully: /zhome/4f/d/187167/QIM/projects/2024_DANFIX_147_RAPID-P/analysis/BachelorProject/Data/Downsampled/Cropped/Week2/Week2_30_top/week2_30_top_1_400_400_400_0.1_data_cropped_2.nii.gz
Patch Size for step 3: (400, 400, 400) Coordinates 